# HW3-评分作业

<span style="color:red; font-weight:bold;">此次作业是评分作业，包含有基础题（10分）和选做题（1分）。注意，如果你选择做选做题，那么作业截止时间将比不做选做题要提前。</span>

<span style="color:red; font-weight:bold;">选做题截止时间：3月4日23：59：59；最终截止时间：3月7日23：29：29</span>

<span style="color:red; font-weight:bold;">请将作业命名为 HW3-评分作业+姓名+学号.ipynb, 例如 HW3-评分作业+张三+1000000000.ipynb，并且只需要提交该notebook</span>

## 作业描述
在这次作业中，我们希望构建一个相当简化的选课系统。

- 同学们首先在预选阶段（preselect）向教务提交自己想要上的课程，并且为每门课分配总和上限为99点的意愿点。
- 教务收到提交后验证同学们的提交是否满足学分以及意愿点总和的限制，并将想选某门课的同学加入到该课程的预选学生名单中。
- 预选阶段结束后，在正式选课阶段（select）教务可以使用不同的选课方法（如随机分配、最大意愿点优先等）来为同学们分配课程。

（注意为了简化，同学们所选课程超过其学分上限，或者意愿点总和超过99之后，我们不做处理，直接忽略后面的课程。同时我们认为学生没有重名、课程没有重名。）

这次作业可能会需要同学们花较多的时间在理解代码架构和环境上，实际的代码量并不是很大。可以在开始作业之前，多花一些时间阅读代码，理解整个代码的结构和功能。同时为了结构清晰（以及方便助教出题），我们也没有考虑整体代码的实现效率，所以一些代码的效率问题大家不要在意:）

### 一、课程与学生类的基本实现
在选课系统中，我们一般可以抽象出三个类：课程类（Course）、学生类（Student）和选课系统类（Dean）。
- 课程类（Course）：课程类包含课程的名称、课程的容量等基本属性，同时还包含了预选学生迭代器（preselected_students）以及最终选上课的学生（enrolled_students）两个属性。为了实现不同的预选算法， 预选学生迭代器（preselected_students）可能需要一些数据结构（比如最小堆等），因此我们给Course类传入了一个preselected_students_class，用于实例化预选学生的数据结构。
- 学生类（Student）：学生类包含学生的姓名、学号等基本属性，同时还包含了学生添加课程的方法（add_course）。
- 选课系统类（Dean）：选课系统类是整个选课系统的核心，包含了课程的容器（courses）和学生的容器（students），同时还包含了预选阶段（preselect_course）和正式选课阶段（select_course）两个方法。

In [1]:
class Course:
    def __init__(self, name, code, instructor, credits, capacity, preselected_students_class):
        self.name = name # 课程名称
        self.code = code # 课程代码
        self.instructor = instructor # 授课老师
        self.credits = credits # 学分
        self.capacity = capacity # 课程容量
        self.enrolled_students = [] # 已选学生列表
        self.preselected_students = preselected_students_class() # 元素为(预选学生,所投意愿点)元组、通过append方法加入元素的一个迭代器

    def enroll_student(self, student):
        """添加学生到课程中"""
        # TODO (1')
        # 这个函数首先会根据当前的已选学生列表判断课程是否已满，如果未满则将学生加入该列表，否则抛出异常Exception
        if len(self.enrolled_students) >= self.capacity:
            # print("called")
            raise Exception('Course capacity exceeded')
        else:
            # if student.name == '学生21':
                # print("here:",self)
            self.enrolled_students.append(student)
            student.enrolled_courses.append(self)

    def preselect(self, student, point):
        """添加预选学生到preselected_students数据结构中,以便教务之后按课程抽签选课"""
        self.preselected_students.append((student,point))
        
    def __repr__(self):
        return f'课程名称：{self.name}，课程代码：{self.code}，授课老师：{self.instructor}，学分：{self.credits}，容量：{self.capacity}'

class Student:
    def __init__(self, name, student_id, credit_limit, prefer_courses_name, priority_points):
        self.name = name # 姓名
        self.student_id = student_id # 学号
        self.credit_limit = credit_limit # 学分上限
        self.current_credits = 0 # 当前学分总和
        self.current_preselect_credits = 0 # 当前预选学分总和
        self.current_priority_points = 0 # 当前意愿点总和
        self.prefer_courses_name = prefer_courses_name # 预选课程列表
        self.priority_points = priority_points # 意愿点列表
        self.preselect_courses = [] # 预选课程列表
        self.enrolled_courses = [] # 已选课程列表


    def add_preselect_course(self, course, priority_point):
        """根据学生在preselect阶段所选的课程和在这个课程上分配的意愿点,将课程加入学生的预选课程列表中,并检查学分和意愿点是否超出限制"""
        if self.current_preselect_credits + course.credits > self.credit_limit:
            raise Exception('Credit limit exceeded')
        if self.current_priority_points + priority_point > 99:
            raise Exception('Priority points exceeded')
        self.preselect_courses.append(course)
        self.current_preselect_credits += course.credits
        self.current_priority_points += priority_point
        
    def add_course(self, course):
        """将课程加入学生的已选课程列表中,并更新学生的当前学分总和"""
        self.enrolled_courses.append(course)
        self.current_credits += course.credits
        
    def __repr__(self) -> str:
        return f'姓名：{self.name}，学号：{self.student_id}，学分上限：{self.credit_limit}'

In [2]:
student1 = Student('张三', '001', 10, ['数学', '英语', '物理'], [10, 20, 30])
student2 = Student('李四', '002', 10, ['数学', '英语', '物理'], [10, 20, 30])
student3 = Student('王五', '003', 10, ['数学', '英语', '物理'], [10, 20, 30])
math_course = Course('数学', '001', '王老师', 3, 2, list)
english_course = Course('英语', '002', '李老师', 3, 2, list)
physics_course = Course('物理', '003', '张老师', 3, 2, list)
for student in [student1, student2, student3]:
    for course, point in zip([math_course, english_course, physics_course], student.priority_points):
        student.add_preselect_course(course, point)
math_course.preselected_students = [(student1, 10), (student2, 10), (student3, 10)]
for student,point in math_course.preselected_students:
    try:
        math_course.enroll_student(student)
    except Exception:
        break
assert student1.current_preselect_credits == 9
assert student1.current_priority_points == 60
assert math_course.enrolled_students == [student1,student2]

#### 类的继承
继承Course类，实现一个新的只包含必修课的类，名字叫CompulsoryCourse，并增加一个属性和两个方法：
- 一个方法叫做set_prerequisite(self, course)，用来设置这门课的先修课程。
- 一个方法叫做get_prerequisite(self)，用来获取这门课的先修课程。
- 一个属性叫做prerequisite，用来存储这门课的先修课程。
    - 你也可以试着将prerequisite设置为私有属性，然后通过getter和setter方法来访问和修改它。

In [3]:
class CompulsoryCourse(Course):
    
    def __init__(self, name, code, instructor, credits, capacity, preselected_students_class):
        #TODO (1') 
        Course.__init__(self, name, code, instructor, credits, capacity, preselected_students_class)
        self._prerequisite = ''

    @property
    def prerequisite(self):
        return self._prerequisite

    @prerequisite.setter
    def prerequisite(self, course_name):
        self._prerequisite = course_name

    def set_prerequisite(self, course_name):
        #TODO (1') 
        self.prerequisite = course_name
        
    def get_prerequisite(self):
        #TODO (1') 
        return self.prerequisite

    def __repr__(self):
        if self.prerequisite:
            return f'课程名称：{self.name} ({self.prerequisite}) ，课程代码：{self.code}，授课老师：{self.instructor}，学分：{self.credits}，容量：{self.capacity}'
        else:
            return f'课程名称：{self.name}，课程代码：{self.code}，授课老师：{self.instructor}，学分：{self.credits}，容量：{self.capacity}'


compulsory_course = CompulsoryCourse('Python程序设计', 'COMP1001', '张三', 3, 2, list)
compulsory_course.set_prerequisite('C程序设计')
assert compulsory_course.get_prerequisite() == 'C程序设计'
assert str(compulsory_course) == "课程名称：Python程序设计 (C程序设计) ，课程代码：COMP1001，授课老师：张三，学分：3，容量：2"

以下是选课系统（Dean）类的基本实现。它首先在初始化时分别读取了课程和学生的信息，然后实现了预选阶段（preselect_course）和正式选课阶段（select_course）两个方法。
- 课程和学生的信息读取：
  - json文件是python（以及很多其它语言中）经常使用的文件格式。一个json可以使用json.load(file_discripter)的方法读取，返回的是一个字典。更多关于json使用的信息可以参考[这里](https://stackoverflow.com/questions/7771011/how-can-i-parse-read-and-use-json-in-python)。
  - 课程信息存储在courses.json文件中，格式如下：
  ```json
  [
    {"课程号": 614205, "课程名": "软件测试导论", "课程类别": "限选", "教师": "郝丹(教授)", "限数": 8, "学分": 3},
  ]
  ```
  - 学生信息存储在students_select_course.json文件中，格式如下：
  ```json
  [
    {"学生编号": 1, "学生名字": "学生1", "想选课程": "太极拳;随机算法;神经网络的计算基础;图神经网络", "意愿点": "19;19;2;59", "学分上限": 10},
  ]
  ```
  - 可以注意到"想选课程"和"意愿点"都是使用分号分隔的字符串，你需要将其分割成列表。意愿点也需要从字符串被转换成整数。
- 预选阶段（preselect_course）
  - 选课系统遍历每个学生的每门所选课程，将该学生加入到课程的预选学生名单中。
- 选课阶段
  - 选课系统遍历每门课程，根据不同的选课方法（如随机分配、最大意愿点优先等，在preselected_students迭代器中实现），遍历preselected_students迭代器，将输出的学生加入到课程中，直到迭代完成或课程抛出课程容量已满的异常

In [4]:
import json

class Dean:
    def __init__(self, preselected_students_class):
        self.courses = []
        self.students = []
        self.course_name_map = {}
        self.student_name_map = {}
        self.preselected_students_class = preselected_students_class # 预选学生迭代器的类型，用于在Dean类中创建课程实例时传递给Course类
        
        self.load_courses()
        self.load_students()

    def load_courses(self):
        # 使用json加载courses.json文件，将课程信息转换为Course类的实例，存储在self.courses列表中
        # 同时将课程名和课程实例的映射关系存储在self.course_name_map字典中
        with open('./courses.json', 'r') as file:
            courses = json.load(file)
        self.courses = [Course(course['课程名'], course['课程号'], course['教师'], course['学分'], course['限数'], self.preselected_students_class) for course in courses]
        self.course_name_map = {course.name:course for course in self.courses}

    def load_students(self):
        # TODO (1')
        # 使用json加载students_select_course.json文件，将学生信息转换为Student类的实例，存储在self.students列表中
        # 同时将学生名和学生实例的映射关系存储在self.student_name_map字典中
        # 注意：students_select_course.json文件中的想选课程和意愿点是以字符串的形式存储的，需要转换为列表，每个意愿点需要转换成整数。
        with open('./students_select_course.json', 'r') as file:
            students = json.load(file)
        self.students = [Student(student['学生名字'], student['学生编号'], student['学分上限'], student['想选课程'].split(';'), 
                                 [int(points) for points in student['意愿点'].split(';')]) for student in students]
        self.student_name_map = {student.name:student for student in self.students}

    def get_by_course_name(self, course_name):
        # TODO (1')
        # 根据课程名返回对应的课程实例
        # print(self.course_name_map[course_name])
        return self.course_name_map[course_name]


    def get_by_student_name(self, student_name):
        # TODO (1')
        # 根据学生名返回对应的学生实例
        return self.student_name_map[student_name]

    def preselect_course(self):
        """根据学生的想选课程和意愿点，将学生加入到课程的预选学生列表中。对每个学生的每门课程，首先需要根据学生的想选课程找到对应的课程实例，然后将学生加入到课程的预选学生列表中。"""
        for student in self.students:
            # 使用zip函数将存储在student实例属性中的学生的想选课程和意愿点进行打包，遍历每个学生的想选课程和意愿点
            for course_name, point in zip(student.prefer_courses_name, student.priority_points):
                course = self.get_by_course_name(course_name)
                # print(course, point)
                student.add_preselect_course(course, point)
                course.preselect(student,point)
            
    def select_course(self):
        # TODO（可能比较难） (1')
        # 遍历每门课程的预选学生迭代器，将学生加入到课程的已选学生列表中，直到课程满员或者迭代器迭代完成
        for course in self.courses:
            # print(course)
            for preselect_student, points in course.preselected_students:
                # if preselect_student.name == '学生21':
                #     print(preselect_student)
                #     print(course)
                #     print(len(course.enrolled_students))
                try:
                    course.enroll_student(preselect_student)
                except Exception: #课程满员
                    break


In [5]:
dean = Dean(list)
assert dean.courses[3].name == "神经网络的计算基础"
assert dean.students[6].name == "学生7"
assert dean.get_by_course_name('神经网络的计算基础').name == '神经网络的计算基础'
assert dean.get_by_student_name('学生7').name == '学生7'
dean.preselect_course()
dean.select_course()
# print(dean.courses[3])
# print(dean.courses[3].enrolled_students[5])
assert dean.courses[3].enrolled_students[5].name == "学生40"
# print(dean.students[20])
# print(dean.students[20].enrolled_courses)
assert dean.students[20].enrolled_courses[0].name == "信息论"

## 二、实现预选学生迭代器
我们希望实现两种不同的预选学生迭代器，分别对应不同的预选算法。
1. 每个学生都有完全相同的概率选到该课程
2. 对每个学生在该课上投的意愿点多少进行排序，意愿点高的先得到课程

### 1. 示例：随机的预选学生迭代器

In [6]:
import random
class RandomList:
    def __init__(self):
        self.lst = []
        
    def append(self, item):
        self.lst.append(item)

    def __iter__(self):
        return self

    def __next__(self):
        if not self.lst:
            raise StopIteration
        return self.lst.pop(random.randint(0, len(self.lst) - 1))

### 2. 最大意愿点优先的预选学生迭代器
在这个迭代器中，我们希望对每个学生在该课上投的意愿点多少进行排序，意愿点高的先得到课程。这件事情可以通过一个最小堆来实现。下面给出了一个最小堆的简单实现，你需要补充的是delete_min方法，它应该能够删除并返回堆中的最小元素，以及__iter__与__next__方法，使其成为一个迭代器。关于最小堆这一数据结构的更多信息可以参考[这里](https://www.digitalocean.com/community/tutorials/min-heap-binary-tree)。

你也可以自行构建功能相同但实现方法不同的迭代器，只要它能够实现最大意愿点优先的结果即可。注意为了方便测试，最好还是将其命名MinHeap。

In [7]:
# 迭代器的构建与使用，以及使用类建立数据结构
class MinHeap:
    def __init__(self, compare = lambda x, y: x[1] < y[1]):
        # 初始化
        self.heap_list = [("",0)] # 堆从1开始
        self.current_size = 0
        self.compare = compare

    def append(self, k):
        # 将k放入最小堆中，同时维护最小堆
        self.heap_list.append(k)
        self.current_size += 1
        self.sift_up(self.current_size)

    def sift_up(self, i):
        # 向上移动二叉树中的值，维护最小堆
        while i // 2 > 0:
            if self.compare(self.heap_list[i], self.heap_list[i // 2]):
                self.heap_list[i], self.heap_list[i // 2] = self.heap_list[i // 2], self.heap_list[i]
            i = i // 2

    def sift_down(self, i):
        # 向下移动二叉树中的值，维护最小堆
        while (i * 2) <= self.current_size:
            mc = self.min_child(i)
            if self.compare(self.heap_list[mc], self.heap_list[i]):
                self.heap_list[i], self.heap_list[mc] = self.heap_list[mc], self.heap_list[i]
            i = mc

    def min_child(self, i):
        if (i * 2) + 1 > self.current_size:
            return i * 2
        else:
            if self.compare(self.heap_list[i * 2], self.heap_list[(i * 2) + 1]):
                return i * 2
            else:
                return (i * 2) + 1

    def delete_min(self):
        # 删除最小值
        if len(self.heap_list) == 1:
            return 'Empty heap'
    
        root = self.heap_list[1]
        self.heap_list[1] = self.heap_list[self.current_size]
        *self.heap_list, _ = self.heap_list
        self.current_size -= 1
        
        # TODO 这里应该做什么？ (1')
        # raise NotImplementedError
        self.sift_down(1)
        return root

    def __iter__(self):
        #TODO (1')
        return self
    
    def __next__(self):
        #TODO (1')
        if self.current_size == 0:
            raise StopIteration
        return self.delete_min()


In [8]:
minheap = MinHeap()
minheap.append(('a', 10))
minheap.append(('b', 20))
minheap.append(('c', 5))
# print(list(minheap))
assert list(minheap) == [('c', 5), ('a', 10), ('b', 20)]

In [9]:
dean = Dean(MinHeap)
dean.preselect_course()
dean.select_course()
assert dean.courses[0].enrolled_students[0].name == "学生25"
assert dean.students[0].enrolled_courses[0].name == "神经网络的计算基础"

## 选做题：***选做一个即可，不重复计分，两个都做按照正确的计分***

### 选做题1：课程的拓扑排序
对于一些课程，有先修课程的要求，例如，要学习微积分，需要先学习数学分析。请实现一个拓扑排序算法，对于给定的课程有向图，输出这个图的拓扑排序结果。如果这个图有环，那么输出None。

In [10]:
class Node:
    def __init__(self, value):
        self.value = value

class Graph:
    def __init__(self):
        self.graph = {}
        
    def add_edge(self, node, neighbor):
        if node not in self.graph:
            self.graph[node] = [neighbor]
        else:
            self.graph[node].append(neighbor)
        if neighbor not in self.graph:
            self.graph[neighbor] = []

# A指向B说明A应当比B先修
def topological_sort(graph):
    # TODO
    in_degree = {}
    ans = []
    for node in graph: # 初始化in_degree
        if node not in in_degree:
            in_degree[node] = 0
        for neighbor in graph[node]:
            if neighbor not in in_degree:
                in_degree[neighbor] = 0
            in_degree[neighbor] = in_degree[neighbor] + 1
    while len(in_degree) > 0:
        if 0 not in in_degree.values(): # 所有node均有后继，图中必有环
            print('None')
            return
        else:
            for node in in_degree:
                if in_degree[node] == 0:
                    break
            ans.append(node)
            for neighbor in graph[node]:
                in_degree[neighbor] = in_degree[neighbor] - 1
            del in_degree[node]
    return ans

In [11]:
nodes = [Node('C程序设计'), Node('Python程序设计'), Node('数据结构'), Node('算法设计'), Node('数据库'), Node('操作系统'), Node('计算机网络')]
graph = Graph()
graph.add_edge(nodes[0], nodes[1])
graph.add_edge(nodes[0], nodes[2])
graph.add_edge(nodes[2], nodes[1])
graph.add_edge(nodes[2], nodes[3])
graph.add_edge(nodes[2], nodes[4])
graph.add_edge(nodes[5], nodes[3])
graph.add_edge(nodes[3], nodes[6])
graph.add_edge(nodes[4], nodes[6])
graph.add_edge(nodes[5], nodes[6])
def validate_answer(graph, answer):
    answer_index = {node.value: index for index, node in enumerate(answer)}
    for node in graph:
        for neighbor in graph[node]:
            if answer_index[node.value] >= answer_index[neighbor.value]:
                return False
    return True
assert validate_answer(graph.graph, topological_sort(graph.graph))

### 选做题2：改进的废弃功能提醒
    对废弃功能提醒装饰器进行如下的改进：同时打印本函数的名字,并接受自定义提醒语句(具体格式为：`f"Function {func.__name__} is deprecated, {custom_message}"`)。函数的名字可以通过调用`f.__name__`获得。
    注意：函数的元信息应当被保存，比如在外部调用f.__doc__时，应当返回原函数的文档字符串等。这项功能可以通过`functools`中与wrap相关的方法实现（推荐，可以参考https://docs.python.org/3/library/functools.html），或者也可以尝试自己实现（不推荐）。


举例：
```python
@deprecate('use f1 instead')
def f(): 
    pass

>>> f()
Function f is deprecated, use f1 instead
>>> f()
>>> f()
```



In [12]:
import functools
def deprecate(custom_message):
    called_cnt = 0
    def decorator(func):
        @functools.wraps(func)
        def my_func(farg, *args, **kwargs):
            # """Test doc string"""
            nonlocal called_cnt # 通过外部环境变量called_cnt记录被调用次数，以实现只在第一次调用时提醒
            if called_cnt == 0:
                print(f"Function {func.__name__} is deprecated, {custom_message}")
            called_cnt += 1
        return my_func
    return decorator
        
# 应用装饰器
@deprecate('use f1 instead')
def f(farg, *args, **kwargs):
    """This is the doc string of the function"""
    print(farg, args, kwargs)
# print(f.__doc__)
assert f.__doc__ == "This is the doc string of the function"
f(1, 2, 3, a=1, b=2, c=3)
f(1, 2, 3, a=1, b=2, c=3)

Function f is deprecated, use f1 instead
